## DATA cleaning

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from gensim.models import Word2Vec
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import csv
import regex as re
import re
import string
import pennylane as qml
import pandas as pd
from sklearn.preprocessing import LabelEncoder


## Read Data

In [30]:
df = pd.read_excel('Train_Task_B_Updated.xlsx')

In [31]:
df.head(10)

,Id,Tweet,Hate,Target,Severity
0,5723,@sudhirchaudhary Abhi tak 2000 ke note me mujh...,0,NaN,NaN
1,2338,@Gulamane_raza @MustakimRazvi Abe katiye tumse...,1,O,H
2,1579,@RajatSharmaLive Ye sab sazish hai....bina sam...,1,O,M
3,6524,abe jao tum to dasko pahle hi fash gye the jab...,1,I,L
4,7618,Ab ye afbah kaun faila Raha hai ki Shahhen bag...,0,NaN,NaN
5,5077,@FilmyKhichdii Nahi sudhar sakta tu.... Waise ...,1,R,L
6,5342,Hindenburg ko America ke 2 Bank ki gadbadi nah...,1,I,L
7,1451,@AcharyaPramodk Agar tum maan lo ki tabligi ja...,1,O,L
8,4975,"@TweetAbhishekA Inme se kahiyoun ke pati, pita...",1,O,L
9,7108,..magar accident mein toh na jane kitne muslim...,0,NaN,NaN


In [32]:
df = df.drop(columns='Id')

In [33]:
df.columns

Index(['Tweet', 'Hate', 'Target', 'Severity'], dtype='object')

In [34]:
df.Hate.unique()

array([0, 1], dtype=int64)

## Preprocessing

In [35]:
# function to clean all data
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    try:
        text = re.sub('(\&amp\;)', '', text)
        # removing any usernames
        text = re.sub('(@[^\s]+)', '', text)
        # removing any hashtags
        text = re.sub('(#[^\s]+)', '', text)
        
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
        # remove `rt` for retweet
        text = re.sub('(rt)', '', text)
        # string.punctuation is a string of all punctuation marks
        # so this gets rid of all punctuation
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
        # getting rid of `httptco`
        text = re.sub('(httptco)', '', text)
    except:
        text = "None"

    return text

round1 = lambda x: clean_text_round1(x)

In [36]:
df['Tweet'] = df.Tweet.astype('str')
df.Tweet

0       @sudhirchaudhary Abhi tak 2000 ke note me mujh...
1       @Gulamane_raza @MustakimRazvi Abe katiye tumse...
2       @RajatSharmaLive Ye sab sazish hai....bina sam...
3       abe jao tum to dasko pahle hi fash gye the jab...
4       Ab ye afbah kaun faila Raha hai ki Shahhen bag...
                              ...                        
6391    @team_hyv Pathaan part 2 bhi aa rha hai, boyco...
6392    @amitku1047 @bordiasanjay @izeenatrana @RahulG...
6393    @HusainSahadal Sachaee ladaee hoti h hamare de...
6394                  Hahaha hahaha kejurdin ððð
6395    Aswini upadhyay aap desh ko grhyudh ke raste p...
Name: Tweet, Length: 6396, dtype: object

In [37]:
df['clean_tweets'] = df['Tweet'].apply(round1)

In [38]:
df

,Tweet,Hate,Target,Severity,clean_tweets
0,@sudhirchaudhary Abhi tak 2000 ke note me mujh...,0,NaN,NaN,abhi tak 2000 ke note me mujhe gps nano chip ...
1,@Gulamane_raza @MustakimRazvi Abe katiye tumse...,1,O,H,abe katiye tumse kuch huaa toh jata nahi bas...
2,@RajatSharmaLive Ye sab sazish hai....bina sam...,1,O,M,ye sab sazish haibina saman ke koi kaise apne...
3,abe jao tum to dasko pahle hi fash gye the jab...,1,I,L,abe jao tum to dasko pahle hi fash gye the jab...
4,Ab ye afbah kaun faila Raha hai ki Shahhen bag...,0,NaN,NaN,ab ye afbah kaun faila raha hai ki shahhen bag...
...,...,...,...,...,...
6391,"@team_hyv Pathaan part 2 bhi aa rha hai, boyco...",1,O,M,pathaan pa 2 bhi aa rha hai boycott karna nhi...
6392,@amitku1047 @bordiasanjay @izeenatrana @RahulG...,1,O,L,hindenburg adani pe kuchh bolta hai tto ke...
6393,@HusainSahadal Sachaee ladaee hoti h hamare de...,1,O,L,sachaee ladaee hoti h hamare desh me tablighi...
6394,Hahaha hahaha kejurdin ððð,0,NaN,NaN,hahaha hahaha kejurdin


In [39]:
def tokenise(sentence):
    # Split the sentence into words based on whitespace
    words = sentence.split()
    return words

In [40]:
df["tokens"] = df["clean_tweets"].apply(tokenise)

In [41]:
df["tokens"]

0       [abhi, tak, 2000, ke, note, me, mujhe, gps, na...
1       [abe, katiye, tumse, kuch, huaa, toh, jata, na...
2       [ye, sab, sazish, haibina, saman, ke, koi, kai...
3       [abe, jao, tum, to, dasko, pahle, hi, fash, gy...
4       [ab, ye, afbah, kaun, faila, raha, hai, ki, sh...
                              ...                        
6391    [pathaan, pa, 2, bhi, aa, rha, hai, boycott, k...
6392    [hindenburg, adani, pe, kuchh, bolta, hai, tto...
6393    [sachaee, ladaee, hoti, h, hamare, desh, me, t...
6394                           [hahaha, hahaha, kejurdin]
6395    [aswini, upadhyay, aap, desh, ko, grhyudh, ke,...
Name: tokens, Length: 6396, dtype: object

### Label Encoding

In [42]:
# Fit and transform the data to get labels
# Initialize LabelEncoder
label_encoder1 = LabelEncoder()

targets = df['Target']

print(targets.unique())
data_labels = label_encoder1.fit_transform(targets)
num_classes = len(np.unique(data_labels))

# Convert to one-hot encoding
one_hot_vectors_targets = np.eye(num_classes)[data_labels]
df['Target_labels'] = data_labels
df['Target_class'] = list(one_hot_vectors_targets)

[nan 'O' 'I' 'R']


In [43]:
label_encoder = LabelEncoder()

sev = df['Severity']

print(sev.unique())
data_labels = label_encoder.fit_transform(sev)
num_classes = len(np.unique(data_labels))

# Convert to one-hot encoding
one_hot_vectors_sev = np.eye(num_classes)[data_labels]
df['Sev_labels'] = data_labels
df['Sev_class'] = list(one_hot_vectors_sev)

[nan 'H' 'M' 'L']


In [44]:
df

,Tweet,Hate,Target,Severity,clean_tweets,tokens,Target_labels,Target_class,Sev_labels,Sev_class
0,@sudhirchaudhary Abhi tak 2000 ke note me mujh...,0,NaN,NaN,abhi tak 2000 ke note me mujhe gps nano chip ...,"[abhi, tak, 2000, ke, note, me, mujhe, gps, na...",3,"[0.0, 0.0, 0.0, 1.0]",3,"[0.0, 0.0, 0.0, 1.0]"
1,@Gulamane_raza @MustakimRazvi Abe katiye tumse...,1,O,H,abe katiye tumse kuch huaa toh jata nahi bas...,"[abe, katiye, tumse, kuch, huaa, toh, jata, na...",1,"[0.0, 1.0, 0.0, 0.0]",0,"[1.0, 0.0, 0.0, 0.0]"
2,@RajatSharmaLive Ye sab sazish hai....bina sam...,1,O,M,ye sab sazish haibina saman ke koi kaise apne...,"[ye, sab, sazish, haibina, saman, ke, koi, kai...",1,"[0.0, 1.0, 0.0, 0.0]",2,"[0.0, 0.0, 1.0, 0.0]"
3,abe jao tum to dasko pahle hi fash gye the jab...,1,I,L,abe jao tum to dasko pahle hi fash gye the jab...,"[abe, jao, tum, to, dasko, pahle, hi, fash, gy...",0,"[1.0, 0.0, 0.0, 0.0]",1,"[0.0, 1.0, 0.0, 0.0]"
4,Ab ye afbah kaun faila Raha hai ki Shahhen bag...,0,NaN,NaN,ab ye afbah kaun faila raha hai ki shahhen bag...,"[ab, ye, afbah, kaun, faila, raha, hai, ki, sh...",3,"[0.0, 0.0, 0.0, 1.0]",3,"[0.0, 0.0, 0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...
6391,"@team_hyv Pathaan part 2 bhi aa rha hai, boyco...",1,O,M,pathaan pa 2 bhi aa rha hai boycott karna nhi...,"[pathaan, pa, 2, bhi, aa, rha, hai, boycott, k...",1,"[0.0, 1.0, 0.0, 0.0]",2,"[0.0, 0.0, 1.0, 0.0]"
6392,@amitku1047 @bordiasanjay @izeenatrana @RahulG...,1,O,L,hindenburg adani pe kuchh bolta hai tto ke...,"[hindenburg, adani, pe, kuchh, bolta, hai, tto...",1,"[0.0, 1.0, 0.0, 0.0]",1,"[0.0, 1.0, 0.0, 0.0]"
6393,@HusainSahadal Sachaee ladaee hoti h hamare de...,1,O,L,sachaee ladaee hoti h hamare desh me tablighi...,"[sachaee, ladaee, hoti, h, hamare, desh, me, t...",1,"[0.0, 1.0, 0.0, 0.0]",1,"[0.0, 1.0, 0.0, 0.0]"
6394,Hahaha hahaha kejurdin ððð,0,NaN,NaN,hahaha hahaha kejurdin,"[hahaha, hahaha, kejurdin]",3,"[0.0, 0.0, 0.0, 1.0]",3,"[0.0, 0.0, 0.0, 1.0]"


### Creating Word2Vec models

In [45]:
w2v_model = Word2Vec(df['tokens'], vector_size=150, window=5, min_count=1, workers=4)

In [46]:
word_vector = w2v_model.wv.most_similar("muslim", topn=10)
print(word_vector)

[('ham', 0.9993175864219666), ('kia', 0.9990960359573364), ('nahi', 0.9990660548210144), ('fir', 0.9989789128303528), ('he', 0.998894989490509), ('yahi', 0.9988822340965271), ('kaam', 0.9988290667533875), ('mar', 0.9988179206848145), ('q', 0.9987779259681702), ('toh', 0.9987019300460815)]


In [47]:
w2v_model.save("word2vec100.model")

In [48]:
def sentence_vector(sentence, model):
    # Tokenize the sentence into words
    words = tokenise(sentence)
    
    # Filter out words that are not in the Word2Vec vocabulary
    words = [word for word in words if word in model.wv.key_to_index]
    
    # If no words in the sentence are in the vocabulary, return a zero vector
    if not words:
        return np.zeros(model.vector_size)
    
    # Compute the mean of the word vectors for each word in the sentence
    vector = np.mean([model.wv[word] for word in words], axis=0)
    return vector

In [49]:
df["word2vec"] = df['clean_tweets'].apply(lambda x: sentence_vector(x,w2v_model))

In [50]:
df['word2vec']

0       [0.04157704, -0.2028364, 0.025532717, -0.24434...
1       [0.113576554, -0.27857405, 0.024261054, 0.0052...
2       [0.12748031, -0.3067371, 0.029009657, -0.00356...
3       [0.104581885, -0.27558458, 0.013990153, 0.0135...
4       [0.11322723, -0.29805982, 0.024223026, -0.0016...
                              ...                        
6391    [0.08445614, -0.23639458, 0.008503209, 0.01276...
6392    [0.11129607, -0.27897233, 0.0065248986, 0.0044...
6393    [0.14881738, -0.36316997, 0.053687613, 0.00054...
6394    [0.004829488, -0.02540317, 0.0033556037, -0.00...
6395    [0.12675522, -0.31474626, 0.016188122, 0.01364...
Name: word2vec, Length: 6396, dtype: object

In [51]:
df.isnull().sum()

Tweet               0
Hate                0
Target           2295
Severity         2295
clean_tweets        0
tokens              0
Target_labels       0
Target_class        0
Sev_labels          0
Sev_class           0
word2vec            0
dtype: int64

## Modeling

### Quantum Nodes

In [52]:
n_qubits = 6

In [53]:
class QNN_model(nn.Module):
    def __init__(self, input_size, n_qubits, output_size):
        super(QNN_model, self).__init__()

        # Fully connected layers
        self.fc2 = nn.Linear(input_size, input_size // 10)
        self.fc3 = nn.Linear(input_size // 10, input_size // 20)
        self.fc4 = nn.Linear(input_size // 20, n_qubits)

        # Output layers for target and severity
        self.target = nn.Linear(n_qubits, output_size)
        self.severity = nn.Linear(n_qubits, output_size)

    def forward(self, x):
        # Pass through fully connected layers with ReLU activation
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        # Output layers (return raw logits)
        x_severity = self.severity(x)  # Shape: [batch_size, output_size]
        x_target = self.target(x)      # Shape: [batch_size, output_size]

        return x_target, x_severity


In [54]:
Qmodel = QNN_model(150,n_qubits,4)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6396 entries, 0 to 6395
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet          6396 non-null   object
 1   Hate           6396 non-null   int64 
 2   Target         4101 non-null   object
 3   Severity       4101 non-null   object
 4   clean_tweets   6396 non-null   object
 5   tokens         6396 non-null   object
 6   Target_labels  6396 non-null   int32 
 7   Target_class   6396 non-null   object
 8   Sev_labels     6396 non-null   int32 
 9   Sev_class      6396 non-null   object
 10  word2vec       6396 non-null   object
dtypes: int32(2), int64(1), object(8)
memory usage: 499.8+ KB


In [56]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Define a custom Dataset class
class Word2VecDataset2(Dataset):
    def __init__(self, dataframe, target_class, target_class2):
        """
        Custom Dataset for Word2Vec inputs with multi-target labels.

        Args:
        - dataframe (pd.DataFrame): Input data containing word2vec vectors and target labels.
        - target_class (str): Column name for the primary target.
        - target_class2 (str): Column name for the secondary target.
        """
        self.data = dataframe
        self.target = target_class
        self.severity = target_class2

        # Validate columns
        required_columns = ['word2vec', self.target, self.severity]
        for col in required_columns:
            if col not in dataframe.columns:
                raise ValueError(f"Missing required column: {col}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Extract the input vector and labels
        vector = torch.tensor(self.data.iloc[index]['word2vec'], dtype=torch.float32)
        label_tar = torch.tensor(self.data.iloc[index][self.target], dtype=torch.float32)  # Classification targets
        label_sev = torch.tensor(self.data.iloc[index][self.severity], dtype=torch.float32)
        return vector, label_tar, label_sev

# Create the dataset and dataloader
target1 = "Target_class"  # Replace with your column name for target
target2 = "Sev_class"     # Replace with your column name for severity
dataset = Word2VecDataset2(df, target1, target2)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Example usage
for vectors, labels1, labels2 in dataloader:
    print("Batch of vectors (shape):", vectors.shape)
    print("Batch of target labels:", labels1.shape)
    print("Batch of severity labels:", labels2.shape)
    break


Batch of vectors (shape): torch.Size([16, 150])
Batch of target labels: torch.Size([16, 4])
Batch of severity labels: torch.Size([16, 4])


In [57]:
device = 'cpu'

In [58]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import csv

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Qmodel.parameters(), lr=0.0005)

filename = 'results_target.csv'

# Write CSV Header
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Epoch', 'Loss', 'Target Accuracy', 'Severity Accuracy'])

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    Qmodel.train()  # Set the model to training mode
    running_loss = 0.0
    correct_tar = 0
    correct_sev = 0
    total = 0

    for inputs, labels_tar, labels_sev in tqdm(dataloader):
        # Move data to the appropriate device (e.g., CPU/GPU)
        inputs, labels_tar, labels_sev = inputs.to(device), labels_tar.to(device), labels_sev.to(device)

        # Forward pass
        outputs_tar, outputs_sev = Qmodel(inputs)
        # print(outputs_sev.shape)
        loss1 = criterion(outputs_tar, labels_tar)
        loss2 = criterion(outputs_sev, labels_sev)
        loss = loss1 + loss2

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update running loss
        running_loss += loss.item() * inputs.size(0)

        # Predictions and accuracy for each target
        # labels_tar = labels_tar.view(-1)  # Shape: [batch_size]
        # labels_sev = labels_sev.view(-1)  # Shape: [batch_size]
        _, labels_tar = torch.max(labels_tar, dim=1)
        _, labels_sev = torch.max(labels_sev, dim=1)

        # Predictions
        _, predicted_tar = torch.max(outputs_tar, dim=1)
        _, predicted_sev = torch.max(outputs_sev, dim=1)

        # Update correct counts
        correct_tar += (predicted_tar == labels_tar).sum().item()
        correct_sev += (predicted_sev == labels_sev).sum().item()
        total += labels_tar.size(0)
    # Epoch metrics
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc_tar = correct_tar / total
    epoch_acc_sev = correct_sev / total

    # Print epoch statistics
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, "
          f"Target Accuracy: {epoch_acc_tar:.4f}, Severity Accuracy: {epoch_acc_sev:.4f}")

    # Log metrics to CSV
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([epoch + 1, epoch_loss, epoch_acc_tar, epoch_acc_sev])


100%|██████████| 400/400 [00:01<00:00, 243.08it/s]


Epoch 1/20, Loss: 2.6205, Target Accuracy: 0.3723, Severity Accuracy: 0.3129


100%|██████████| 400/400 [00:01<00:00, 275.52it/s]


Epoch 2/20, Loss: 2.5387, Target Accuracy: 0.4285, Severity Accuracy: 0.3532


100%|██████████| 400/400 [00:01<00:00, 289.71it/s]


...

Epoch 18/20, Loss: 2.2953, Target Accuracy: 0.5400, Severity Accuracy: 0.4551


100%|██████████| 400/400 [00:03<00:00, 128.37it/s]


Epoch 19/20, Loss: 2.2929, Target Accuracy: 0.5438, Severity Accuracy: 0.4531


100%|██████████| 400/400 [00:01<00:00, 294.57it/s]

Epoch 20/20, Loss: 2.2930, Target Accuracy: 0.5413, Severity Accuracy: 0.4545


In [59]:
test_data = pd.read_excel("Test_Task_B (1).xlsx")

In [60]:
test_data.head(5)

,Id,Tweet,Target,Severity
0,1516,"100 +crore hindu+ others ghar me band h, leki...",O,M
1,3359,jo bhosadiwale shaheen bagh khali nahi kara pa...,I,L
2,6706,ak bar itihas par lena bhavi jaanÃ¢ÂÂ¤Ã¯Â¸Â,NaN,NaN
3,6472,Le hindu :- ye kab huaÃ°ÂÂÂ,NaN,NaN
4,4174,#SupremeCourt Hamare Sath Aisa kab tak Hota ra...,NaN,NaN


In [61]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        800 non-null    int64 
 1   Tweet     800 non-null    object
 2   Target    513 non-null    object
 3   Severity  513 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.1+ KB


In [62]:
test_data['temp'] = test_data['Tweet'].apply(lambda x: clean_text_round1(x))
test_data['word2vec'] = test_data['temp'].apply(lambda x: sentence_vector(x,w2v_model)) 
test_data.head(4)

,Id,Tweet,Target,Severity,temp,word2vec
0,1516,"100 +crore hindu+ others ghar me band h, leki...",O,M,100 crore hindu others ghar me band h lekin y...,"[0.12360205, -0.31104943, 0.033026326, -0.0010..."
1,3359,jo bhosadiwale shaheen bagh khali nahi kara pa...,I,L,jo bhosadiwale shaheen bagh khali nahi kara pa...,"[0.124218434, -0.319708, -0.007396103, -0.0068..."
2,6706,ak bar itihas par lena bhavi jaanÃ¢ÂÂ¤Ã¯Â¸Â,NaN,NaN,ak bar itihas par lena bhavi jaan,"[0.065638, -0.17999119, 0.0074263825, -0.00442..."
3,6472,Le hindu :- ye kab huaÃ°ÂÂÂ,NaN,NaN,le hindu ye kab hua,"[0.1498843, -0.4096803, 0.014262569, 0.0213173..."


In [63]:
target_map = {'I': 1, 'O': 2, 'R': 3}
test_data['Target'] = test_data['Target'].map(target_map)

# Encode Severity column
severity_map = {'L': 1, 'M': 2, 'H': 3}
test_data['Severity'] = test_data['Severity'].map(severity_map)

# Replace NaN in Target and Severity where Hate == 0
test_data['Target'] = test_data['Target'].fillna(0)
test_data['Severity'] = test_data['Severity'].fillna(0)

In [64]:
test_data

,Id,Tweet,Target,Severity,temp,word2vec
0,1516,"100 +crore hindu+ others ghar me band h, leki...",2.0,2.0,100 crore hindu others ghar me band h lekin y...,"[0.12360205, -0.31104943, 0.033026326, -0.0010..."
1,3359,jo bhosadiwale shaheen bagh khali nahi kara pa...,1.0,1.0,jo bhosadiwale shaheen bagh khali nahi kara pa...,"[0.124218434, -0.319708, -0.007396103, -0.0068..."
2,6706,ak bar itihas par lena bhavi jaanÃ¢ÂÂ¤Ã¯Â¸Â,0.0,0.0,ak bar itihas par lena bhavi jaan,"[0.065638, -0.17999119, 0.0074263825, -0.00442..."
3,6472,Le hindu :- ye kab huaÃ°ÂÂÂ,0.0,0.0,le hindu ye kab hua,"[0.1498843, -0.4096803, 0.014262569, 0.0213173..."
4,4174,#SupremeCourt Hamare Sath Aisa kab tak Hota ra...,0.0,0.0,hamare sath aisa kab tak hota rahega kab tak ...,"[0.09396923, -0.2608282, 0.0034924608, 0.00445..."
...,...,...,...,...,...,...
795,7512,Dekho gt ne rcb ko haraya tha uska badla dhoni...,2.0,1.0,dekho gt ne rcb ko haraya tha uska badla dhoni...,"[0.12949915, -0.3259424, 0.016945528, 0.004634..."
796,7015,Jarur koi dukhi hoga feku ujjawala yojna KaÃ°Â...,0.0,0.0,jarur koi dukhi hoga feku ujjawala yojna ka,"[0.07544893, -0.21427032, 0.0028452242, 0.0018..."
797,5410,Jaag Muslim Kab Jaagay Ga? #hijab #Mannat #Kar...,0.0,0.0,jaag muslim kab jaagay ga httpstcoq5y...,"[0.08968107, -0.24038458, 0.0113143325, 0.0200..."
798,7954,Manish Kashyap ko jail mai dalo ð¡ð¡ð¡ð¡,1.0,1.0,manish kashyap ko jail mai dalo,"[0.09195539, -0.21902128, 0.008752491, -0.0085..."


In [65]:

class TestDataset(Dataset):
    def __init__(self, dataframe):
        """
        Custom Dataset for Word2Vec inputs with multi-target labels.

        Args:
        - dataframe (pd.DataFrame): Input data containing word2vec vectors and target labels.
        """
        self.data = dataframe
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # Extract the input vector and labels
        vector = torch.tensor(self.data.iloc[index]['word2vec'], dtype=torch.float32)
        return vector


In [66]:
dataset_test = TestDataset(test_data)
Sev_outputs = [] 
target_outputs = []
for i in range(dataset_test.__len__()):
    input = dataset_test[i]  # Get the input data
    input = input.unsqueeze(0)  # Add batch dimension (1, features) if needed
    outputs_tar, outputs_sev = Qmodel(input)  # Get softmax output
    binary_pred = torch.argmax(outputs_tar, dim=-1).item()  
    binary_pred2 = torch.argmax(outputs_sev, dim=-1).item()  
    target_outputs.append(binary_pred)
    Sev_outputs.append(binary_pred2)


In [67]:
test_data['Target_pred'] = target_outputs
test_data['Severity_pred'] = Sev_outputs

### Predictions

In [68]:
from sklearn.metrics import classification_report

def predict_and_evaluate(test_data):
    """
    Evaluates the performance of a multi-label classification model on test data.
    
    Args:
    - test_data (pd.DataFrame): DataFrame containing the ground truth labels ('Hate', 'Fake') 
                                and the predicted labels ('Hate_pred', 'Fake_pred').

    Returns:
    - None: Prints classification reports for each target column.
    """
    required_columns = ['Target', 'Severity', 'Target_pred', 'Severity_pred']
    missing_columns = [col for col in required_columns if col not in test_data.columns]
    if missing_columns:
        raise KeyError(f"The following required columns are missing in the dataset: {missing_columns}")
    
    # Replace NaN in Target and Severity where Hate == 0
    test_data['Target'] = test_data['Target'].fillna(0)
    test_data['Severity'] = test_data['Severity'].fillna(0)
    
    # Extract true labels and predicted labels
    y_test = test_data[['Target', 'Severity']].astype(int)  # Convert to integers if not already
    y_pred = test_data[['Target_pred', 'Severity_pred']].astype(int)
    
    # Evaluate each target separately
    for column in y_test.columns:
        print(f"\nClassification Report for {column}:")
        print(classification_report(y_test[column], y_pred[f"{column}_pred"]))


In [69]:
predict_and_evaluate(test_data)


Classification Report for Target:
              precision    recall  f1-score   support

           0       0.32      0.10      0.15       287
           1       0.02      0.05      0.03       132
           2       0.00      0.00      0.00       294
           3       0.16      0.70      0.27        87

    accuracy                           0.12       800
   macro avg       0.13      0.21      0.11       800
weighted avg       0.14      0.12      0.09       800


Classification Report for Severity:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       287
           1       0.52      0.25      0.34       255
           2       0.42      0.69      0.52       190
           3       0.09      0.50      0.16        68

    accuracy                           0.29       800
   macro avg       0.26      0.36      0.25       800
weighted avg       0.27      0.29      0.24       800



c:\Users\naman\OneDrive\Documents\Speed-test\lulc\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\naman\OneDrive\Documents\Speed-test\lulc\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\naman\OneDrive\Documents\Speed-test\lulc\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr